<a href="https://colab.research.google.com/github/voldrek/Meus_projetos/blob/main/RomaneioVersion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
from openpyxl import load_workbook

In [ ]:
import pandas as pd

# Carregar o arquivo CSV usando pandas com codificação 'latin1'
csv_filename = 'teste.csv'
data = pd.read_csv(csv_filename, encoding='latin1')

# Especificar o nome do arquivo Excel de saída
excel_filename = 'saida.xlsx'

# Salvar os dados no formato Excel
data.to_excel(excel_filename, index=False)


In [247]:
df = pd.read_excel('ola.xlsx')
#Filtrando os Excluir e deletando da tabela
filtro_excluir_novas = df['Ação'].str.lower().isin(['excluir', 'novas'])

# Filtra o DataFrame para manter apenas as linhas que não atendem ao filtro acima
df = df[~filtro_excluir_novas]
#Aplicando os filtros dos cabeçalhos extras
filtroProduto = df['Produto'].str.contains('Produto', case=False)
linhasProduto = df[filtroProduto]
colunaProduto = df['Produto']
indiceLinhasProduto = linhasProduto.index
df = df.drop(index=indiceLinhasProduto)

df['Service Id'] = df['Service Id'].apply(lambda x: '{:.0f}'.format(x))

# Variáveis para padrões de números para extração
padrao_celular = r'(\b\d{11}\b)'
padrao_iccd = r'\b\d{20}\b'
padrao_imei = r'(\b\d{15}\b)'
padrao_nf = r'(\b\d{8}\b)'
# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    service_data = row['Service Id']
    telefone = re.search(padrao_celular, service_data)
    iccd = re.search(padrao_iccd, service_data)
    imei = re.search(padrao_imei, service_data)
    if telefone:
        df.at[index, 'Numero telefone'] = telefone.group(0)
    if iccd:
        if 'ICCD' in df.columns:
            df.at[index, 'ICCD'] = iccd.group(0)
        else:
            df['ICCD'] = None
            df.at[index, 'ICCD'] = iccd.group(0)
    if imei:
        df.at[index, 'IMEI'] = imei.group(0)

with pd.ExcelWriter('resposta_romaneio.xlsx') as writer:
    df2 = pd.DataFrame().to_excel(writer, sheet_name='Planilha1', index=False)

    if 'ICCD' in df.columns:
      numero_iccd = df['ICCD']
      limpeza_iccd = numero_iccd.dropna()
      limpeza_iccd.to_excel(writer, sheet_name='Planilha1', startcol=0, index=False)  # Coluna 'ICCD' na coluna A
    if 'Numero telefone' in df.columns:
      numero_telefone = df['Numero telefone']
      limpeza_telefone = numero_telefone.dropna()
      limpeza_telefone.to_excel(writer, sheet_name='Planilha1', startcol=1, index=False)  # Coluna 'Telefone' na coluna b
    if 'Ordem de vendas' in df.columns:
      numero_ordem = df['Ordem de vendas']
      limpeza_ordem = numero_ordem.dropna()
      limpeza_ordem.to_excel(writer, sheet_name='Planilha1', startcol=2, index=False)  # Coluna 'Ordem de Venda' na coluna c
    if 'Nota Fiscal' in df.columns:
      numero_nf = df['Nota Fiscal']
      limpeza_nf = numero_nf.dropna()
      limpeza_nf.to_excel(writer, sheet_name='Planilha1', startcol=3, index=False) # Coluna 'Nota Fiscal' na coluna c
    if 'IMEI' in df.columns:
      numero_imei = df['IMEI']
      limpeza_imei = numero_imei.dropna()
      limpeza_imei.to_excel(writer, sheet_name='Planilha1', startcol=4, index=False)  # Coluna 'IMEI' na coluna E
    if 'Conta de serviço' in df.columns:
      numero_conta = df['Conta de serviço'].dropna()
      numero_conta.to_excel(writer, sheet_name='Planilha1', startcol=5, index=False)

df2 = pd.read_excel('resposta_romaneio.xlsx')
df2 = df2.dropna(subset=['ICCD'])
df2.to_excel('resposta_romaneio.xlsx')

In [ ]:

df2.head(5)

In [ ]:
df2.info()